Create base dataset csv file with columns: language, speaker, audio_raw_data.

Loads first the wav files and decodes them into a pandas dataframe. Each audio is approximately 60s long

In [8]:
#import libraries
import numpy as np
from scipy.io import wavfile
import os
import pandas as pd

sample_length = 60 * 16000 # 16kHz sampling of 60 seconds of audio

# print size of list of audio files
# folder = os.getcwd() + "/../rec_train/"
# folder = os.getcwd() + "/../rec_test/test_known_speakers/"
folder = os.getcwd() + "/../rec_test/test_unheard_speakers/"

# list of wav files in the folder
audio_files_list = [audio for audio in os.listdir(folder) if audio.endswith(".wav")]
print(len(audio_files_list))

# memorize audio recordings and thei associated language and speaker
# audio_recs = np.zeros( (len(audio_files_list), sample_length), dtype="int16")
audio_recs = []
languages = []
speakers = []

# create starting point: dataset with all the audio recordings
for i, audio in enumerate(audio_files_list):
    sample_rate, audio_rec = wavfile.read(folder + audio)
    print("audio = ", audio, audio_rec.shape)
    audio_recs.append(audio_rec)
    languages.append(audio[4:7])
    speakers.append(audio[8 : len(audio) - 7])

# print unique speakers and languages
print(np.unique(speakers))
print(np.unique(languages))


# Create a dictionary with the structured data
data_dictionary = {
    'language': languages,
    'speaker': speakers,
    'audio_raw_data': audio_recs
}

'''
# Specify the data types for each column
data_types = {
    'audio_raw_data': 'object',
    'language': 'str'
}
'''
dataset = pd.DataFrame(data_dictionary)
# Save the DataFrame to a CSV file
#df.to_csv('dataset0.csv', index=False)

print(dataset.head())


42
audio =  rec_eng_documentary_03.wav (959488,)
audio =  rec_fra_interview_8.wav (957696,)
audio =  rec_fra_interview_0.wav (960000,)
audio =  rec_fra_lessons_02.wav (955136,)
audio =  rec_ita_syne_01.wav (957440,)
audio =  rec_fra_lessons_01.wav (955136,)
audio =  rec_eng_youtube_03.wav (957440,)
audio =  rec_eng_lesson_01.wav (957696,)
audio =  rec_eng_bourbon_01.wav (957696,)
audio =  rec_ita_genitore1_01.wav (957440,)
audio =  rec_fra_lorenzo1_04.wav (959488,)
audio =  rec_eng_documentary_02.wav (957440,)
audio =  rec_fra_interview_2.wav (957696,)
audio =  rec_fra_lessons_07.wav (957440,)
audio =  rec_ita_genitore2_01.wav (957696,)
audio =  rec_eng_documentary_01.wav (957440,)
audio =  rec_fra_lessons_04.wav (957696,)
audio =  rec_fra_lessons_03.wav (954880,)
audio =  rec_eng_lesson_02.wav (957696,)
audio =  rec_ita_rita_02.wav (957696,)
audio =  rec_fra_interview_7.wav (959744,)
audio =  rec_ita_rita_01.wav (957696,)
audio =  rec_eng_documentary_05.wav (957696,)
audio =  rec_eng_

## Training - Validation - Test splitting criteria

Create training and validation dataset by splitting the data based on the speaker and language:

- all the speakers having 1 or 2 languages associated will have their data split 75-25 between training and validation. Each audio sample is split in a piece of 45 seconds (used for training) and another separate piece of 15 seconds.
- speakers having more than 2 languages associated will have 75% of languages in training and 25% of languages in validation. Each audio sample here is taken entirely without splitting, and placed in the corresponding dataset.

The data collected are audio samples of 60 seconds. The split between training and validation is done in such a way to have completely separated data frames between the sets. 
This way the audio frames computed will never overlap between training and validation. This is important in order to have validation data that is unseen in training set. T
his is due to the frames of the audio samples computed in sequences of frame_size with some overlap (hop_size). 
In order to have zero overlap between frames in the 2 datasets, we divide an audio samples in 2 separate non-overlapping pieces, where the frames are computed.

This way the validation set can be used to estimate:
- known speakers speaking in languages that have been already heard from them
- known speakers speaking in languages that were never heard from them -> useful to understand quality of model's knowledge

The test set will be created ad-hoc when a lot of data is collected. A few separate test sets will be created, needed to evaluate the performance of the model in different scenarios:

1. known speakers in heard languages -> evaluate model performance in tested scenarios
2. known speakers in un-heard languages -> evaluate performance for recognizing language instead of the speaker vocal characteristics
3. unknown speakers -> evaluate performance for recognizing language from an unseen speaker

The forecast for the test task is having the performance in the case 3 being lower than the case 2. Having separate test sets is useful in order to have an unbiased estimate of the model's performance on different tasks.

In [2]:
window = 5.6 # seconds of audio in input
hop = 1.875 # overlapping window time in seconds
frequency = 16000


This function takes as input a series of audio samples, and extracts frames from it without any 75%-25% splitting

In [3]:

def split_100(audio_list: pd.DataFrame) -> (pd.DataFrame):
    audio_splits = []
    languages_test = []
    speakers_test = []
    
    samples = audio_list["audio_raw_data"].to_numpy()
    languages = audio_list["language"].to_numpy()
    speakers = audio_list["speaker"].to_numpy()
    
    for i, sample in enumerate(samples):
        
        for time in range(0, int((60-window) * frequency), int(hop * frequency)):
            audio_splits.append( sample[time : int(time + window * frequency) ].tolist() )
            languages_test.append(languages[i])
            speakers_test.append(speakers[i])

    data_dictionary = {
        'language': languages_test,
        'speaker': speakers_test,
        'audio_raw_data': audio_splits
    }

    return pd.DataFrame(data_dictionary)


Code for splitting audio samples in 2 sequences of 45 - 15 seconds each. The frames extracted from the audio samples are well separated and there are no overlapping frames between the 2 splits

In [4]:

def split_75(audio_list: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
	audio_splits_train = []
	audio_splits_val = []
	languages_train = []
	languages_val = []
	speakers_train = []
	speakers_val = []

	samples = audio_list["audio_raw_data"].to_numpy()
	languages = audio_list["language"].to_numpy()
	speakers = audio_list["speaker"].to_numpy()

	for i, sample in enumerate(samples):

		for time in range(0, int((45-window) * frequency), int(hop * frequency)):
			audio_splits_train.append(sample[time : int(time + window * frequency) ].tolist() )
			languages_train.append(languages[i])
			speakers_train.append(speakers[i])
		
		for time in range(45 * frequency, int((60-window) * frequency), int(hop * frequency)):
			audio_splits_val.append(sample[time : int(time + window * frequency) ].tolist() )
			languages_val.append(languages[i])
			speakers_val.append(speakers[i])

	data_dictionary_train = {
		'language': languages_train,
		'speaker': speakers_train,
		'audio_raw_data': audio_splits_train
	}

	data_dictionary_val = {
		'language': languages_val,
		'speaker': speakers_val,
		'audio_raw_data': audio_splits_val
	}
	
	# specify type of data for each column in the dictionaries
	df_train = pd.DataFrame(data_dictionary_train)
	#df_train = df_train.astype({"language": str, "speaker": str, "audio_raw_data": object})
	df_val = pd.DataFrame(data_dictionary_val)
	#df_val = df_val.astype({"language": str, "speaker": str, "audio_raw_data": object})

	return df_train, df_val



Print the list of speakers found in the rec folder

In [9]:

split_ratio = 0.75 # 75% of training, 25% of validation

classes_list = ["ita", "eng", "fra"] # substitute with np.unique(languages) to obtain whole set of languages
speakers_list = np.unique(speakers)
print(speakers_list)


['bourbon' 'documentary' 'genitore1' 'genitore2' 'geopop' 'intervie'
 'jakidale' 'lesson' 'lessons' 'lorenzo1' 'pdvg' 'realscience' 'rita'
 'syne' 'youtube']


Select a subset of the languages present in the dataset for training. The resulting dataframe contains only the chosen languages

In [10]:
# select the samples that are in dataset_windowed and whose language is in classes_list
valid_samples = []
for sample in dataset.iterrows():
    if sample[1]['language'] in classes_list:
        valid_samples.append(sample[1])

valid_samples = pd.DataFrame(valid_samples)

print("samples considered: ", len(valid_samples))
print("entire dataset: ", dataset.shape)

samples considered:  42
entire dataset:  (42, 3)


Splits the dataset into training and validation using the criteria mentioned above

In [7]:
dataset_train = pd.DataFrame(columns=["language", "speaker", "audio_raw_data"])
dataset_validation = pd.DataFrame(columns=["language", "speaker", "audio_raw_data"])

for speaker in speakers_list:
	# take slice of the dataset containing the samples associated to one speaker
	data_speaker = valid_samples[ valid_samples["speaker"] == speaker]
	print("speaker: ", speaker, " speaks: ", np.unique(data_speaker["language"]) )
	print("samples quantity: ", data_speaker.shape)

	# compute number of languages spoken by the speaker
	langs_spoken = np.unique(data_speaker["language"])
	print("langs_spoken: ", langs_spoken)
	langs_spoken_count = langs_spoken.shape[0]
	
	samples_number = len(data_speaker)


	# split an audio sequence of 60 seconds into one sequence of 45s and one sequence of 15s
	# such that none of the frames in the sequences are shared between the two splits
	# the 45s sequence goes in training, the 15s sequence goes in validation

	data_speaker_train, data_speaker_val = split_75(data_speaker)
	
	count_train = data_speaker_train.shape[0]
	count_valid = data_speaker_val.shape[0]

	print("added ", count_train, " samples to training")
	print("added ", count_valid, " samples to validation")

	dataset_train = pd.concat([dataset_train, data_speaker_train])
	dataset_validation = pd.concat([dataset_validation, data_speaker_val])


len_train = len(dataset_train)
len_valid = len(dataset_validation)
total = len_train + len_valid
print("dataset_train: ", len_train)
print("dataset_validation: ", len_valid)
print("training ratio: ", len_train / total * 100.0, "%")
print("validation ratio: ", len_valid / total * 100.0, "%")
print("total number of samples: ", total)

speaker:  adrian  speaks:  ['ita']
samples quantity:  (2, 3)
langs_spoken:  ['ita']
added  44  samples to training
added  12  samples to validation
speaker:  alessandro  speaks:  ['eng' 'ita']
samples quantity:  (5, 3)
langs_spoken:  ['eng' 'ita']
added  110  samples to training
added  30  samples to validation
speaker:  andrea  speaks:  ['fra']
samples quantity:  (1, 3)
langs_spoken:  ['fra']
added  22  samples to training
added  6  samples to validation
speaker:  bob  speaks:  ['ita']
samples quantity:  (1, 3)
langs_spoken:  ['ita']
added  22  samples to training
added  6  samples to validation
speaker:  chiara  speaks:  ['eng']
samples quantity:  (3, 3)
langs_spoken:  ['eng']
added  66  samples to training
added  18  samples to validation
speaker:  claudio  speaks:  ['eng' 'ita']
samples quantity:  (5, 3)
langs_spoken:  ['eng' 'ita']
added  110  samples to training
added  30  samples to validation
speaker:  elena  speaks:  ['eng' 'ita']
samples quantity:  (7, 3)
langs_spoken:  ['eng

Saves the training and validation dataset in memory

In [8]:

folder = os.path.dirname(os.getcwd()) + "/datasets/"
dataset_train.to_csv(folder + "dataset_train.csv", index=False)
dataset_validation.to_csv(folder + "dataset_validation.csv", index=False)

The resulting csv datasets computed are processed in order to get the MFCCs, which are the actual features of the model.

The MFCCs are computed with a cpp script, both at inference time on the arduino and at training time with a linux machine

### Test sets computation

In [11]:
dataset_test = pd.DataFrame(columns=["language", "speaker", "audio_raw_data"])

for speaker in speakers_list:
	# take slice of the dataset containing the samples associated to one speaker
	data_speaker = valid_samples[ valid_samples["speaker"] == speaker]
	print("speaker: ", speaker, " speaks: ", np.unique(data_speaker["language"]) )
	print("samples quantity: ", data_speaker.shape)

	# compute number of languages spoken by the speaker
	langs_spoken = np.unique(data_speaker["language"])
	print("langs_spoken: ", langs_spoken)
	langs_spoken_count = langs_spoken.shape[0]
	
	samples_number = len(data_speaker)

	# this version of the code is meant to compute the test set, so all the generated samples are added
	# to the test set. The splitting is 100% test set and 0% training set

	data_speaker_test = split_100(data_speaker)
	
	count_test = data_speaker_test.shape[0]

	print("added ", count_test, " samples to this test set")

	dataset_test = pd.concat([dataset_test, data_speaker_test])

len_test = len(dataset_test)
print("dataset_test size: ", len_test)

speaker:  bourbon  speaks:  ['eng']
samples quantity:  (1, 3)
langs_spoken:  ['eng']
added  30  samples to this test set
speaker:  documentary  speaks:  ['eng' 'ita']
samples quantity:  (7, 3)
langs_spoken:  ['eng' 'ita']
added  210  samples to this test set
speaker:  genitore1  speaks:  ['ita']
samples quantity:  (2, 3)
langs_spoken:  ['ita']
added  60  samples to this test set
speaker:  genitore2  speaks:  ['ita']
samples quantity:  (2, 3)
langs_spoken:  ['ita']
added  60  samples to this test set
speaker:  geopop  speaks:  ['ita']
samples quantity:  (2, 3)
langs_spoken:  ['ita']
added  60  samples to this test set
speaker:  intervie  speaks:  ['fra']
samples quantity:  (5, 3)
langs_spoken:  ['fra']
added  150  samples to this test set
speaker:  jakidale  speaks:  ['ita']
samples quantity:  (1, 3)
langs_spoken:  ['ita']
added  30  samples to this test set
speaker:  lesson  speaks:  ['eng']
samples quantity:  (2, 3)
langs_spoken:  ['eng']
added  60  samples to this test set
speaker:  

In [12]:
folder = os.path.dirname(os.getcwd()) + "/datasets/"
#dataset_test.to_csv(folder + "dataset_test_known_speakers.csv", index=False)
dataset_test.to_csv(folder + "dataset_test_unheard_speakers.csv", index=False)